In [ ]:
# 1. Import library
import requests
import pandas as pd
import os
import time

In [ ]:
# 3. Define base url dan api key
API_KEY = "YOUR API KEYBASE_URL = "https://api.openweathermap.org/data/2.5/weather"

In [ ]:
# 3. DAFTAR KOTA YANG AKAN KITA AMBIL DATANYA
# Kita bisa menambahkan sebanyak mungkin kota di sini.
cities = [
    "Jakarta,ID", "New York,US", "London,GB", "Tokyo,JP", "Paris,FR",
    "Sydney,AU", "Mumbai,IN", "Beijing,CN", "Cairo,EG", "Moscow,RU",
    "Rio de Janeiro,BR", "Toronto,CA", "Berlin,DE", "Madrid,ES", "Rome,IT",
    
]

In [ ]:
def get_weather_data(city_query):
    """
    Mengambil data cuaca untuk satu kota dari OpenWeatherMap.
    """
    params = {
        "q": city_query,
        "appid": API_KEY,
        "units": "metric"  # Gunakan Celsius
    }
    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()  # Cek error
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f"Error HTTP untuk kota {city_query}: {http_err}")
        return None
    except Exception as err:
        print(f"Error lainnya untuk kota {city_query}: {err}")
        return None

In [ ]:
# 5. PROSES EKSTRAK UNTUK SEMUA KOTA
all_weather_data = []
print("Memulai pengambilan data cuaca untuk semua kota...")

for city in cities:
    print(f"-> Mengambil data untuk: {city}")
    data = get_weather_data(city)
    
    # Jika data berhasil diambil dan tidak ada pesan error
    if data and "cod" in data and data["cod"] == 200:
        all_weather_data.append(data)
    
    # Jeda singkat untuk menghormati rate limit API (60 panggilan/menit)
    time.sleep(1) 

print(f"\nSelesai! Berhasil mengambil data untuk {len(all_weather_data)} kota.")

Memulai pengambilan data cuaca untuk semua kota...
-> Mengambil data untuk: Jakarta,ID
-> Mengambil data untuk: New York,US
-> Mengambil data untuk: London,GB
-> Mengambil data untuk: Tokyo,JP
-> Mengambil data untuk: Paris,FR
-> Mengambil data untuk: Sydney,AU
-> Mengambil data untuk: Mumbai,IN
-> Mengambil data untuk: Beijing,CN
-> Mengambil data untuk: Cairo,EG
-> Mengambil data untuk: Moscow,RU
-> Mengambil data untuk: Rio de Janeiro,BR
-> Mengambil data untuk: Toronto,CA
-> Mengambil data untuk: Berlin,DE
-> Mengambil data untuk: Madrid,ES
-> Mengambil data untuk: Rome,IT

Selesai! Berhasil mengambil data untuk 15 kota.


`cities`: Berisi daftar nama kota yang akan diproses satu per satu. Formatnya seperti *NamaKota,KodeNegara* (contoh: `Jakarta,ID`).

`get_weather_data(city_query)`: Ini adalah fungsi sederhana yang dibuat agar kodenya lebih rapi. Fungsinya menerima satu nama kota, lalu mengambil dan mengembalikan data cuacanya.

`for city in cities:` Digunakan untuk melakukan perulangan atau looping. Kode di dalamnya akan dijalankan berulang kali untuk setiap kota yang ada di daftar *cities*.

`all_weather_data.append(data)`: Jika ada data cuaca dari satu kota berhasil diambil, hasilnya akan ditambahkan ke dalam list *all_weather_data*.

`time.sleep(1)`: Digunakan untuk memberi jeda 1 detik setiap kali memanggil API. Ini penting karena layanan API gratis biasanya punya batas jumlah permintaan per detik. Dengan jeda ini, kita tidak akan melewati batas dan terhindar dari pemblokiran.


In [ ]:
# Pastikan kita punya data
if all_weather_data:
    
    # 1. UBAH DATA MENJADI DATAFRAME
    # data JSON yang (nested) menjadi tabel data frame.
    df = pd.json_normalize(all_weather_data)
    
    # 2. PILIH KOLOM YANG PENTING SAJA
    # Dari puluhan kolom, kita hanya butuh beberapa yang relevan.
    # Nama kolom ini didapat dari melihat struktur JSON atau df.columns
    columns_to_keep = [
        'name', 'sys.country', 'main.temp', 'main.feels_like', 'main.pressure',
        'main.humidity', 'weather', 'wind.speed', 'dt'
    ]
    df = df[columns_to_keep]
    
    # 3. BERSIHKAN KOLOM 'weather'
    # Kolom 'weather' berisi list of dictionary. Kita ambil deskripsinya saja.
    # Kita akan membuat kolom baru 'weather_description'
    df['weather_description'] = df['weather'].apply(lambda x: x[0]['description'])
    df.drop(columns=['weather'], inplace=True) # Menghapus kolom lama
    
    # 4. RENAMING KOLOM
    df.rename(columns={
        'name': 'city',
        'sys.country': 'country',
        'main.temp': 'temperature_celsius',
        'main.feels_like': 'feels_like_celsius',
        'main.pressure': 'pressure_hpa',
        'main.humidity': 'humidity_percent',
        'wind.speed': 'wind_speed_ms',
        'dt': 'unix_timestamp'
    }, inplace=True)
    
    # 5. UBAH TIPE DATA
    # 'unix_timestamp' diubah menjadi tanggal yang bisa dibaca manusia
    df['date'] = pd.to_datetime(df['unix_timestamp'], unit='s')
    
    # Ubah kolom lainnya ke tipe numerik
    numeric_cols = ['temperature_celsius', 'feels_like_celsius', 'pressure_hpa', 'humidity_percent', 'wind_speed_ms']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col])
        
    # 6. SUSUN ULANG URUTAN KOLOM
    final_columns = [
        'city', 'country', 'date', 'temperature_celsius', 'feels_like_celsius',
        'weather_description', 'pressure_hpa', 'humidity_percent', 'wind_speed_ms'
    ]
    df = df[final_columns]
    
    # Tampilkan hasil
    print("Data berhasil ditransform!")
    print("\n--- 5 Data Teratas ---")
    print(df.head())
    print(f"\nTotal baris data: {len(df)}")
    print("\nInfo Tipe Data:")
    df.info()

else:
    print("Transformasi dibatalkan karena tidak ada data yang valid.")


Data berhasil ditransform!

--- 5 Data Teratas ---
       city country                date  temperature_celsius  \
0   Jakarta      ID 2025-10-24 07:33:38                28.80   
1  New York      US 2025-10-24 07:35:37                 9.23   
2    London      GB 2025-10-24 07:27:35                 6.19   
3     Tokyo      JP 2025-10-24 07:35:03                15.65   
4     Paris      FR 2025-10-24 07:30:05                 6.04   

   feels_like_celsius          weather_description  pressure_hpa  \
0               33.59                   light rain          1007   
1                7.26                    clear sky          1017   
2                2.15                    clear sky          1000   
3               15.10  light intensity shower rain          1025   
4                2.81                    clear sky          1008   

   humidity_percent  wind_speed_ms  
0                77           4.63  
1                71           3.58  
2                86           6.69  
3      

`pd.json_normalize(all_weather_data)` dipakai untuk mengubah data berbentuk list JSON jadi tabel (DataFrame). Fungsi ini juga bisa otomatis membuka data yang berada di dalam bagian lain, misalnya seperti `main.temp` atau `sys.country`.

`df[columns_to_keep]`: Digunakan untuk memilih kolom-kolom yang penting saja, agar tabel jadi lebih rapi dan mudah dibaca.

`df['weather'].apply(...)`: Kolom **weather** berisi data seperti `[{"id": 804, "main": "Clouds", "description": "overcast clouds", "icon": "04d"}]`. Dari situ kita hanya ambil bagian *description*-nya (contohnya “overcast clouds”). Fungsi `apply` dengan `lambda` membantu mengambil data itu satu per satu di setiap baris.

`pd.to_datetime(df['unix_timestamp'], unit='s')`: Data waktu dari API biasanya dalam bentuk **Unix Timestamp** (jumlah detik sejak tahun 1970). Kode ini mengubahnya jadi format tanggal dan jam yang lebih mudah dibaca oleh kita.

`df.info()`: Dipakai untuk mengecek apakah setiap kolom di DataFrame sudah punya tipe data yang sesuai dan untuk melihat summary isi tabel.


In [ ]:
# Pastikan DataFrame 'df' ada dari langkah sebelumnya
if 'df' in locals() and not df.empty:
    
    # 1. KONFIGURASI KONEKSI BIGQUERY
    project_id = "weather-porto2"     
    dataset_id = "weather_data"               
    table_id = "global_current_weather"       
    
    # Path ke file kunci JSON yang unduh
    key_path = "E:\Purwadhika\Porto 2\service_account_peramal_cuaca.json"  
    
    # Set environment variable untuk autentikasi
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = key_path
    
    # 2. IMPOR LIBRARY BIGQUERY
    from google.cloud import bigquery
    
    # 3. INISIALISASI KLIEN BIGQUERY
    client = bigquery.Client(project=project_id)
    
    # 4. DEFINISIKAN TUJUAN TABEL
    table_ref = f"{project_id}.{dataset_id}.{table_id}"
    
    # 5. KONFIGURASI PROSES LOAD
    # WRITE_TRUNCATE: hapus dan timpa tabel lama
    job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
    
    print(f"Mengirim data ke tabel BigQuery: {table_ref}...")
    
    # 6. JALANKAN PROSES LOAD
    try:
        load_job = client.load_table_from_dataframe(
            df, table_ref, job_config=job_config
        )
        load_job.result() 
        print(f"Berhasil! {load_job.output_rows} baris data telah dimuat ke {table_ref}.")
    except Exception as e:
        print(f"Terjadi error saat mengirim data ke BigQuery: {e}")

else:
    print("Pengiriman data dibatalkan karena DataFrame 'df' kosong atau tidak ada.")

Mengirim data ke tabel BigQuery: weather-porto2.weather_data.global_current_weather...
Berhasil! 15 baris data telah dimuat ke weather-porto2.weather_data.global_current_weather.
